- **`fft = FourierDiscriminantMeta()`**
- img `torch.float32` -> fft.get_meta(img) trả ra `torch.complex64` => không pass vào được model torch.nn
- Hiện tại: thử tách real và complex của complex64 rồi gộp lại (3x256x256 complex -> 6x256x256) => kết quả tệ

In [1]:
!pip install -q gdown

In [2]:
import gdown
gdown.download('https://drive.google.com/uc?id=157bjO1_cFuSd0HWDUuAmcHRJDVyWpOxB', "./", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=157bjO1_cFuSd0HWDUuAmcHRJDVyWpOxB
From (redirected): https://drive.google.com/uc?id=157bjO1_cFuSd0HWDUuAmcHRJDVyWpOxB&confirm=t&uuid=0a322c0e-d738-418c-8c84-b89020f30eb8
To: /kaggle/working/LOLdataset.zip
100%|██████████| 347M/347M [00:08<00:00, 40.0MB/s]


'./LOLdataset.zip'

In [3]:
!unzip -q LOLdataset.zip

In [4]:
import sys
sys.path.append('..')
import torchvision.models as models
import os
import argparse
import random
import numpy as np
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms.functional as TF
from torchvision import transforms
from glob import glob
from sklearn.model_selection import train_test_split
import cv2
import gc
import os
import random
import time
import wandb
import matplotlib.pyplot as plt
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from tqdm import tqdm

In [5]:
class Augment_RGB_torch:
    def __init__(self):
        pass
    def transform0(self, torch_tensor):
        return torch_tensor   
    def transform1(self, torch_tensor):
        torch_tensor = torch.rot90(torch_tensor, k=1, dims=[-1,-2])
        return torch_tensor
    def transform2(self, torch_tensor):
        torch_tensor = torch.rot90(torch_tensor, k=2, dims=[-1,-2])
        return torch_tensor
    def transform3(self, torch_tensor):
        torch_tensor = torch.rot90(torch_tensor, k=3, dims=[-1,-2])
        return torch_tensor
    def transform4(self, torch_tensor):
        torch_tensor = torch_tensor.flip(-2)
        return torch_tensor
    def transform5(self, torch_tensor):
        torch_tensor = (torch.rot90(torch_tensor, k=1, dims=[-1,-2])).flip(-2)
        return torch_tensor
    def transform6(self, torch_tensor):
        torch_tensor = (torch.rot90(torch_tensor, k=2, dims=[-1,-2])).flip(-2)
        return torch_tensor
    def transform7(self, torch_tensor):
        torch_tensor = (torch.rot90(torch_tensor, k=3, dims=[-1,-2])).flip(-2)
        return torch_tensor

def load_image(filepath):
    image = Image.open(filepath).convert('RGB')
    image = image.astype(np.float32)
    img = img/255.
    return image

def load_img(filepath):
    img = cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    img = img/255.
    return img

In [6]:
augment = Augment_RGB_torch()
transforms_aug = [method for method in dir(augment) if callable(getattr(augment, method)) if not method.startswith('_')] 
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.RandomResizedCrop(256, scale=(0.8, 1.0)),  # Randomly crop and resize
    transforms.ToTensor(),
])

In [ ]:
import torch
from torchvision.utils import save_image

class FourierDiscriminantMeta(object):
    def __init__(self, shift: bool = True, one_side: bool = False):
        """
        Initialize the FourierDiscriminantMeta class.

        Parameters:
        shift (bool): Whether to shift the zero-frequency component to the center.
        one_side (bool): Whether to compute the one-sided FFT.
        """
        self.shift = shift
        self.one_side = one_side
    
    def get_meta(self, img: torch.Tensor) -> torch.Tensor:
        """
        Compute the Fourier transform of the input image.

        Parameters:
        img (torch.Tensor): 4-D tensor (b,1,h,w) representing the input image.

        Returns:
        torch.Tensor: Fourier transformed image.
        """
        if self.one_side:
            f = torch.fft.rfft2(img, dim=(-2, -1))
        else:
            f = torch.fft.fft2(img, dim=(-2, -1))
        
        if self.shift:
            f = torch.fft.fftshift(f, dim=(-2, -1))
        
        return f
    
    def visualize(self, f: torch.Tensor, save_path: str = None, binarization: bool = False, normalize: bool = False) -> torch.Tensor:
        """
        Visualize the Fourier transformed image.

        Parameters:
        f (torch.Tensor): Fourier transformed image.
        save_path (str): Path to save the visualized image.
        binarization (bool): Whether to binarize the image.
        normalize (bool): Whether to normalize the image.

        Returns:
        torch.Tensor: Visualized image.
        """
        f_img = 20 * torch.log(torch.abs(f))
        if binarization:
            f_img = torch.where(f_img > 0, torch.ones_like(f_img), torch.zeros_like(f_img))
            
        if save_path is not None:
            save_image(f_img, save_path, normalize=normalize)
        
        return f_img
    
    def inverse_meta(self, f: torch.Tensor) -> torch.Tensor:
        """
        Compute the inverse Fourier transform of the input image.

        Parameters:
        f (torch.Tensor): 4-D tensor (b,1,h,w) representing the Fourier transformed image.

        Returns:
        torch.Tensor: Inverse Fourier transformed image.
        """
        if self.shift:
            f = torch.fft.ifftshift(f, dim=(-2, -1))
            
        if self.one_side:
            f = torch.fft.irfft2(f, dim=(-2, -1))
        else:
            f = torch.fft.ifft2(f, dim=(-2, -1))
        
        return torch.abs(f)
    
    def rgb2gray(self, tensor: torch.Tensor) -> torch.Tensor:
        """
        Convert an RGB tensor image to grayscale.

        Parameters:
        tensor (torch.Tensor): 4-D tensor (b,c,h,w) representing the RGB image.

        Returns:
        torch.Tensor: 4-D tensor (b,1,h,w) representing the grayscale image.
        """
        gray = 0.299 * tensor[:, 0, :, :] + 0.587 * tensor[:, 1, :, :] + 0.114 * tensor[:, 2, :, :]
        return gray.unsqueeze(1)

In [8]:
fft = FourierDiscriminantMeta()

In [9]:
def split_real_imag(tensor):
    # Split along the channel dimension
    c = tensor.shape[1] // 2
    real = tensor[:, :c, :, :]  # First half: Real
    imag = tensor[:, c:, :, :]  # Second half: Imaginary
    return real, imag

In [10]:
class GlobalImageDataset(Dataset):
    def __init__(self, options, low_quality_images, high_quality_images, labels, transform=None):
        self.options = options
        self.low_quality_images = low_quality_images
        self.high_quality_images = high_quality_images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.low_quality_images)

    def __getitem__(self, idx):
        low_img_path = self.low_quality_images[idx]
        high_img_path = self.high_quality_images[idx]

        # Load images as numpy arrays
        low_image = load_img(low_img_path)
        high_image = load_img(high_img_path)

        # Convert numpy arrays to PIL images
        low_image = Image.fromarray((low_image * 255).astype(np.uint8))
        high_image = Image.fromarray((high_image * 255).astype(np.uint8))

        # Apply transformations if provided
        if self.transform:
            low_image = self.transform(low_image)
            high_image = self.transform(high_image)

        # Convert to tensors if the transform does not already do so
        if not isinstance(low_image, torch.Tensor):
            low_image = TF.to_tensor(low_image)
        if not isinstance(high_image, torch.Tensor):
            high_image = TF.to_tensor(high_image)

        # crop input and target
        ps = 256
        H = high_image.shape[1]
        W = high_image.shape[2]
        # r = np.random.randint(0, H - ps) if not H-ps else 0
        # c = np.random.randint(0, W - ps) if not H-ps else 0
        if H-ps==0:
            r=0
            c=0
        else:
            r = np.random.randint(0, H - ps)
            c = np.random.randint(0, W - ps)
        low_image = low_image[:, r:r + ps, c:c + ps]
        high_image = high_image[:, r:r + ps, c:c + ps]

        apply_trans = transforms_aug[random.getrandbits(3)]

        low_image = getattr(augment, apply_trans)(low_image)
        high_image = getattr(augment, apply_trans)(high_image)

        # Apply Fourier Transform to both images
        low_image = fft.get_meta(low_image.unsqueeze(0)).squeeze(0)  
        high_image = fft.get_meta(high_image.unsqueeze(0)).squeeze(0)
        
        low_image_real = low_image.real
        low_image_imag = low_image.imag
        
        high_image_real = high_image.real
        high_image_imag = high_image.imag
        
        low_image = torch.cat([low_image_real, low_image_imag], dim=0)
        high_image = torch.cat([high_image_real, high_image_imag], dim=0) 
        
        label = self.labels[idx]
        return low_image, high_image, label


def load_data(options, datasets, transform=None, val_split=0.2):
    all_low_images = []
    all_high_images = []
    all_labels = []

    for data_dir, low_subdir, high_subdir, label in datasets:
        low_image_dir = os.path.join(data_dir, low_subdir)
        high_image_dir = os.path.join(data_dir, high_subdir)

        low_images = [os.path.join(low_image_dir, img) for img in os.listdir(low_image_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]
        high_images = [os.path.join(high_image_dir, img) for img in os.listdir(high_image_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]

        # Ensure the lengths match by only keeping the common elements
        if len(low_images) != len(high_images):
            print(f"Warning: Mismatch in number of images for label {label}. Adjusting to the minimum number of images.")
            min_len = min(len(low_images), len(high_images))
            low_images = low_images[:min_len]
            high_images = high_images[:min_len]

        all_low_images.extend(low_images)
        all_high_images.extend(high_images)
        all_labels.extend([label] * len(low_images))

    # Split data into training and validation sets
    train_low, val_low, train_high, val_high, train_labels, val_labels = train_test_split(
        all_low_images, all_high_images, all_labels, test_size=val_split, random_state=42
    )

    # Create Dataset objects
    train_dataset = GlobalImageDataset(options, train_low, train_high, train_labels, transform=transform)
    val_dataset = GlobalImageDataset(options, val_low, val_high, val_labels, transform=transform)

    return train_dataset, val_dataset

In [11]:
datasets = [("/kaggle/working/our485", "low", "high", 0)]

# Load and split the data
train_dataset, val_dataset = load_data(None, datasets, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# Example of iterating through the train loader
for i, (low_images, high_images, labels) in enumerate(train_loader):
    print(low_images.shape)  # e.g., torch.Size([32, 3, 640, 840])
    print(high_images.shape)  # e.g., torch.Size([32, 3, 640, 840])
    print(labels.shape)  # e.g., torch.Size([32])
    # print(low_images)
    break

torch.Size([32, 6, 256, 256])
torch.Size([32, 6, 256, 256])
torch.Size([32])


### UNet

In [12]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [13]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = (DoubleConv(n_channels, 64))
        self.down1 = (Down(64, 128))
        self.down2 = (Down(128, 256))
        self.down3 = (Down(256, 512))
        factor = 2 if bilinear else 1
        self.down4 = (Down(512, 1024 // factor))
        self.up1 = (Up(1024, 512 // factor, bilinear))
        self.up2 = (Up(512, 256 // factor, bilinear))
        self.up3 = (Up(256, 128 // factor, bilinear))
        self.up4 = (Up(128, 64, bilinear))
        self.outc = (OutConv(64, n_classes))

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

    def use_checkpointing(self):
        self.inc = torch.utils.checkpoint(self.inc)
        self.down1 = torch.utils.checkpoint(self.down1)
        self.down2 = torch.utils.checkpoint(self.down2)
        self.down3 = torch.utils.checkpoint(self.down3)
        self.down4 = torch.utils.checkpoint(self.down4)
        self.up1 = torch.utils.checkpoint(self.up1)
        self.up2 = torch.utils.checkpoint(self.up2)
        self.up3 = torch.utils.checkpoint(self.up3)
        self.up4 = torch.utils.checkpoint(self.up4)
        self.outc = torch.utils.checkpoint(self.outc)

In [14]:
model = UNet(n_channels=6, n_classes=6, bilinear=False)

In [15]:
for i, (low_images, high_images, labels) in enumerate(train_loader):
    print(model(low_images).shape)
    break

torch.Size([32, 6, 256, 256])


In [16]:
# real_part, imag_part = split_real_imag(model(low_images))

# test = fft.inverse_meta(torch.complex(real_part, imag_part))

In [17]:
# test.shape

In [18]:
# test[0]

### Training

In [19]:
NUM_EPOCHS=100
BATCH_SIZE=32
LR=1e-4
STEP_SIZE=20
BETAS = (0.9, 0.999)
EPS=1e-08
GAMMA = 0.67
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
DEVICE

device(type='cuda')

In [21]:
def complex_mse_loss_concat(predicted, target):
    pred_real, pred_imag = split_real_imag(predicted)
    targ_real, targ_imag = split_real_imag(target)

    real_loss = torch.nn.functional.mse_loss(pred_real, targ_real)
    imag_loss = torch.nn.functional.mse_loss(pred_imag, targ_imag)

    return real_loss + imag_loss

In [22]:
class Network:
    def __init__(self, for_inference=False, train_dataloader=train_loader, val_dataloader=val_loader,
                lr=LR, betas=BETAS, step_size = STEP_SIZE, device=DEVICE, num_epochs=NUM_EPOCHS,
                eps = EPS, gamma=GAMMA, run_id=None):
        self.for_inference=for_inference
        self.model = UNet(n_channels=6, n_classes=6, bilinear=False).to(device)
        self.device=device
        
        if not self.for_inference:
            self.loss_fn = complex_mse_loss_concat
            self.optimizer = optim.Adam(self.model.parameters(), lr=lr, betas=betas, eps=eps)
            self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=step_size, gamma=gamma)

            self.train_dataloader = train_dataloader
            self.val_dataloader = val_dataloader
            self.num_epochs = num_epochs

            self.run_id = run_id
            
    def save_network(self, epoch, train_loss, valid_loss, checkpoint_path):
        os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
        checkpoint = {
            'epoch': epoch,
            'train_loss': train_loss,
            'valid_loss': valid_loss,
            'model': self.model.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'learning_rate_scheduler': self.scheduler.state_dict(),
            'network': self
        }
        torch.save(checkpoint, checkpoint_path)
        
    def load_network(self, checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        if not self.for_inference:
            self.model.load_state_dict(checkpoint['model'])
            self.optimizer.load_state_dict(checkpoint['optimizer'])
            self.scheduler.load_state_dict(checkpoint['learning_rate_scheduler'])
        return checkpoint['epoch'], checkpoint['train_loss'], checkpoint['valid_loss']

    def save_model(self, checkpoint_path):
        os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
        torch.save(self.model, checkpoint_path)
    
    def train_step(self, low_images, high_images):
        low_images, high_images = low_images.to(self.device), high_images.to(self.device)
        self.optimizer.zero_grad()
        res_images = self.model(low_images)
        loss = self.loss_fn(res_images, high_images)
        loss.backward()
        self.optimizer.step()
        return loss.item()
    
    def valid_step(self, low_images, high_images):
        low_images, high_images = low_images.to(self.device), high_images.to(self.device)
        res_images = self.model(low_images)
        loss = self.loss_fn(res_images, high_images)
        return loss.item()
    
    def train_model(self):
        last_valid_loss = float('inf')
        wandb.init(project='UNet',
                   resume='allow',
                   config={
                       'lr': LR,
                       'betas': BETAS,
                       'eps': EPS,
                       'step_size': STEP_SIZE,
                       'gamma': GAMMA,
                       'num_epochs': NUM_EPOCHS,
                   },
                   id=self.run_id)
        wandb.watch(self.model)

        for epoch in range(self.num_epochs):
            
            start = time.time()

            # ------------------- TRAIN -------------------
            self.model.train()
            train_epoch_loss = 0

            for low_images, high_images, _  in tqdm(self.train_dataloader, desc=f'Epoch {epoch+1}/{self.num_epochs}'):
                loss = self.train_step(low_images, high_images)
                train_epoch_loss += loss
            
            del high_images
            del low_images
            gc.collect()
            torch.cuda.empty_cache()
            
            train_epoch_loss /= len(self.train_dataloader)

            end = time.time()

            # ------------------- VALID -------------------
            self.model.eval()
            with torch.no_grad():
                valid_epoch_loss = 0

                for low_images, high_images, _ in tqdm(self.val_dataloader, desc=f'Epoch {epoch+1}/{self.num_epochs}'):
                    loss = self.valid_step(low_images, high_images)
                    valid_epoch_loss += loss
                
                del high_images
                del low_images
                gc.collect()
                torch.cuda.empty_cache()
                
                valid_epoch_loss /= len(self.val_dataloader)
            
            # ------------------- LOG -------------------
            wandb.log({
                'train_loss': train_epoch_loss,
                'valid_loss': valid_epoch_loss,
                'learning_rate': self.optimizer.param_groups[0]['lr']
            })
            self.scheduler.step()

            # ------------------- VERBOSE -------------------
            print(f'Epoch {epoch+1}/{self.num_epochs} | Train Loss: {train_epoch_loss:.4f} | Valid Loss: {valid_epoch_loss:.4f} | Time: {end-start:.2f}s')

            # ------------------- CHECKPOINT -------------------
            self.save_network(epoch, train_epoch_loss, valid_epoch_loss, './network_checkpoint_latest.pth')
            self.save_model('./latest/model.pth')
            if valid_epoch_loss < last_valid_loss:
                last_valid_loss = valid_epoch_loss
                self.save_network(epoch, train_epoch_loss, valid_epoch_loss, './network_checkpoint_best.pth')
                self.save_model('./best/model.pth')
                print("New best checkpoint saved!")
        
        wandb.finish()
        
    def inference(self, low_image, high_image=None):
        """
        - lq_image: torch.Tensor
            3D Tensor (C, H, W)
        - hq_image: torch.Tesnor
            3D Tensor (C, H, W). This parameter is optional, for comparing the model output and the 
            ground-truth high-quality image. If used solely for inference, skip this. Default is None/
        """
        low_image_fft = fft.get_meta(low_image.unsqueeze(0))
        
        low_image_real = low_image.real
        low_image_imag = low_image.imag        
        low_image_combined = torch.cat([low_image_real, low_image_imag], dim=0).to(device)

        with torch.no_grad():
            res_image_combined = self.model(low_image_combined)
            
        real_part, imag_part = split_real_imag(res_image_combined)
        res_image = fft.inverse_meta(torch.complex(real_part, imag_part))
        print(">> Size of low-quality image:", low_image.size())
        print(">> Size of restored image:", res_image.size())
                                     
        if high_image != None:
            high_image = high_image.squeeze(0) 
            print(">> Size of high-quality image:", high_image.size())
            fig, axes = plt.subplots(1, 3, figsize=(10, 6))
            axes[0].imshow(low_image.cpu().detach().permute((1, 2, 0)))
            axes[0].set_title('Low Quality')
            axes[1].imshow(res_image.cpu().detach().permute((1, 2, 0)))
            axes[1].set_title('Restored')
            axes[2].imshow(high_image.cpu().detach().permute((1, 2, 0)))
            axes[2].set_title('High Quality')
            for ax in axes.flat:
                ax.axis('off')
        else:
            fig, axes = plt.subplots(1, 2, figsize=(10, 6))
            axes[0].imshow(low_image.cpu().detach().permute((1, 2, 0)))
            axes[0].set_title('Low Quality')
            axes[1].imshow(res_image.cpu().detach().permute((1, 2, 0)))
            axes[1].set_title('Restored')
            for ax in axes.flat:
                ax.axis('off')
        
        plt.tight_layout()        
        plt.show()        
        return res_image    
    
    def train_model_continue(self, checkpoint_path):
        
        start_epoch, train_loss, valid_loss = self.load_network(checkpoint_path)
        print("Previous train loss: ", train_loss)
        print("Previous valid loss: ", valid_loss)
        print("------------------- Resuming training -------------------")
        self.num_epochs -= start_epoch
        self.train_model()

In [ ]:
wandb.login(key="")
unet = Network()
unet.train_model()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: namnguyen2103. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241125_032834-p4d9x5tr
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run electric-water-6
wandb: ⭐️ View project at https://wandb.ai/namnguyen2103/UNet
wandb: 🚀 View run at https://wandb.ai/namnguyen2103/UNet/runs/p4d9x5tr
Epoch 1/100: 100%|██████████| 4/4 [00:03<00:00,  1.15it/s]


Epoch 1/100 | Train Loss: 16123.4382 | Valid Loss: 13847.5059 | Time: 23.47s
New best checkpoint saved!


Epoch 2/100: 100%|██████████| 4/4 [00:03<00:00,  1.17it/s]


Epoch 2/100 | Train Loss: 16303.1055 | Valid Loss: 13722.9531 | Time: 22.53s
New best checkpoint saved!


Epoch 3/100: 100%|██████████| 4/4 [00:03<00:00,  1.15it/s]


...

Epoch 99/100: 100%|██████████| 4/4 [00:03<00:00,  1.11it/s]


Epoch 99/100 | Train Loss: 15395.2933 | Valid Loss: 12813.4691 | Time: 24.66s


Epoch 100/100: 100%|██████████| 4/4 [00:03<00:00,  1.08it/s]


Epoch 100/100 | Train Loss: 15139.6864 | Valid Loss: 12922.7050 | Time: 24.86s


wandb:                                                                                
wandb: 
wandb: Run history:
wandb: learning_rate ██████████▅▅▅▅▅▅▅▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
wandb:    train_loss █▇██▇▆▆▅▇▅▅▄▅▅▄▄▃▄▃▄▅▃▂▃▄▂▃▄▂▃▂▃▂▄▁▁▁▂▁▄
wandb:    valid_loss █▇▇█▇▆▇▇▅▅▅▄▃▄▄▄▄▃▃▄▂▄▄▁▃▁▁▂▃▁▂▁▁▂▁▁▂▂▂▂
wandb: 
wandb: Run summary:
wandb: learning_rate 2e-05
wandb:    train_loss 15139.68637
wandb:    valid_loss 12922.70496
wandb: 
wandb: 🚀 View run electric-water-6 at: https://wandb.ai/namnguyen2103/UNet/runs/p4d9x5tr
wandb: ⭐️ View project at: https://wandb.ai/namnguyen2103/UNet
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20241125_032834-p4d9x5tr/logs
